In [18]:
import random
import math

def split_customer_demand(customer_id, demand, capacity):
    splits = []
    while demand > capacity:
        splits.append((f"{customer_id}p{len(splits)+1}", capacity))
        demand -= capacity
    if demand > 0:
        splits.append((f"{customer_id}p{len(splits)+1}", demand))
    return splits

def get_customer_parts(customers, demands, capacity):
    customer_parts = []
    demand_parts = []
    for c, d in zip(customers, demands):
        if d > capacity:
            splits = split_customer_demand(c, d, capacity)
            for label, part_demand in splits:
                customer_parts.append(label)
                demand_parts.append(part_demand)
        else:
            customer_parts.append(str(c))
            demand_parts.append(d)
    return customer_parts, demand_parts

def build_demands_dict(customers, demands, capacity):
    demands_dict = {}
    for c, d in zip(customers, demands):
        if d > capacity:
            splits = split_customer_demand(c, d, capacity)
            for label, part_demand in splits:
                demands_dict[label] = part_demand
        else:
            demands_dict[str(c)] = d
    return demands_dict

def generate_random_chromosome(customer_parts, demand_parts, vehicles, capacity):
    unassigned = list(zip(customer_parts, demand_parts))
    random.shuffle(unassigned)
    vehicles_left = vehicles[:]
    random.shuffle(vehicles_left)
    chromosome = []
    used_vehicles = set()
    i = 0
    n = len(unassigned)
    while i < n and vehicles_left:
        v = vehicles_left.pop()
        used_vehicles.add(v)
        group = []
        load = 0
        while i < n and load + unassigned[i][1] <= capacity:
            group.append(unassigned[i][0])
            load += unassigned[i][1]
            i += 1
        if group:
            chromosome.extend(group)
            chromosome.append(str(v))
    if i < n:
        return None
    for v in vehicles:
        if v not in used_vehicles:
            chromosome.append(str(v))
    return chromosome

def parse_chromosome(chromosome, vehicles):
    vehicle_set = set(str(v) for v in vehicles)
    assignments = {v: [] for v in vehicles}
    i = 0
    n = len(chromosome)
    while i < n:
        group = []
        while i < n and chromosome[i] not in vehicle_set:
            group.append(chromosome[i])
            i += 1
        if i < n and chromosome[i] in vehicle_set:
            v = int(chromosome[i])
            assignments[v].extend(group)
            i += 1
    return assignments

def is_legitimate_chromosome(chrom, customer_parts, vehicles, demands_dict, capacity):
    assigned_customers = set()
    assignments = parse_chromosome(chrom, vehicles)
    for v, cust_list in assignments.items():
        total_load = 0
        for cust in cust_list:
            if cust not in customer_parts or cust in assigned_customers:
                return False
            assigned_customers.add(cust)
            total_load += demands_dict[cust]
        if total_load > capacity:
            return False
    if assigned_customers != set(customer_parts):
        return False
    if len(chrom) != len(set(chrom)):
        return False
    if not all(str(v) in chrom for v in vehicles):
        return False
    return True

def calculate_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def calculate_vehicle_distances(assignments, customers, demands, vehicles, coordinates, depot=(0, 0)):
    cust_part_coords = {}
    for i, c in enumerate(customers):
        cust_part_coords[str(c)] = coordinates[i]
    for i, c in enumerate(customers):
        for part_num in range(2, 20):
            cust_part_coords[f"{c}p{part_num}"] = coordinates[i]
    vehicle_distances = {}
    for v, cust_list in assignments.items():
        if not cust_list:
            vehicle_distances[v] = 0.0
            continue
        total_dist = 0.0
        prev_point = depot
        for cust in cust_list:
            total_dist += calculate_distance(prev_point, cust_part_coords[cust])
            prev_point = cust_part_coords[cust]
        total_dist += calculate_distance(prev_point, depot)
        vehicle_distances[v] = total_dist
    return vehicle_distances

def single_point_crossover(parent1, parent2):
    length = len(parent1)
    if length < 2:
        return parent1[:], parent2[:]
    point = random.randint(1, length - 2)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def swap_mutation(chrom):
    c = chrom[:]
    idx1, idx2 = random.sample(range(len(c)), 2)
    c[idx1], c[idx2] = c[idx2], c[idx1]
    return c

def repair_chromosome(chrom, customer_parts, vehicles):
    seen = set()
    new_chrom = []
    for gene in chrom:
        if gene not in seen:
            new_chrom.append(gene)
            seen.add(gene)
    all_numbers = set(customer_parts + [str(v) for v in vehicles])
    missing = all_numbers - set(new_chrom)
    new_chrom += list(missing)
    return new_chrom[:len(customer_parts) + len(vehicles)]

def genetic_algorithm():
    # --- User Inputs ---
    num_customers = int(input("Enter the number of customers: "))
    num_vehicles = int(input("Enter the number of vehicles: "))
    customers = []
    demands = []
    coordinates = []
    for i in range(1, num_customers + 1):
        demand = int(input(f"Enter demand for customer {i}: "))
        x = float(input(f"Enter x-coordinate for customer {i}: "))
        y = float(input(f"Enter y-coordinate for customer {i}: "))
        customers.append(i)
        demands.append(demand)
        coordinates.append((x, y))
    vehicles = [i + num_customers for i in range(1, num_vehicles + 1)]
    truck_capacity = int(input("Enter the load carrying capacity of each truck: "))
    population_size = int(input("Enter the population size: "))
    num_generations = int(input("Enter the number of generations: "))

    customer_parts, demand_parts = get_customer_parts(customers, demands, truck_capacity)
    demands_dict = build_demands_dict(customers, demands, truck_capacity)

    # Initialize population
    population = []
    attempts = 0
    while len(population) < population_size and attempts < population_size * 100:
        chrom = generate_random_chromosome(customer_parts, demand_parts, vehicles, truck_capacity)
        if chrom and is_legitimate_chromosome(chrom, customer_parts, vehicles, demands_dict, truck_capacity):
            population.append(chrom)
        attempts += 1

    if len(population) < population_size:
        print("Could not generate enough legitimate chromosomes for initial population.")
        return

    # Evolution
    for gen in range(num_generations):
        next_population = []
        while len(next_population) < population_size:
            parent1, parent2 = random.sample(population, 2)
            child1, child2 = single_point_crossover(parent1, parent2)
            if random.random() < 0.2:
                child1 = swap_mutation(child1)
            if random.random() < 0.2:
                child2 = swap_mutation(child2)
            child1 = repair_chromosome(child1, customer_parts, vehicles)
            child2 = repair_chromosome(child2, customer_parts, vehicles)
            if is_legitimate_chromosome(child1, customer_parts, vehicles, demands_dict, truck_capacity):
                next_population.append(child1)
            else:
                new_chrom = None
                attempts = 0
                while not new_chrom and attempts < 100:
                    new_chrom = generate_random_chromosome(customer_parts, demand_parts, vehicles, truck_capacity)
                    if new_chrom and not is_legitimate_chromosome(new_chrom, customer_parts, vehicles, demands_dict, truck_capacity):
                        new_chrom = None
                    attempts += 1
                if new_chrom:
                    next_population.append(new_chrom)
            if len(next_population) < population_size:
                if is_legitimate_chromosome(child2, customer_parts, vehicles, demands_dict, truck_capacity):
                    next_population.append(child2)
                else:
                    new_chrom = None
                    attempts = 0
                    while not new_chrom and attempts < 100:
                        new_chrom = generate_random_chromosome(customer_parts, demand_parts, vehicles, truck_capacity)
                        if new_chrom and not is_legitimate_chromosome(new_chrom, customer_parts, vehicles, demands_dict, truck_capacity):
                            new_chrom = None
                        attempts += 1
                    if new_chrom:
                        next_population.append(new_chrom)
        population = next_population[:population_size]

    # Evaluate and rank final population
    chromosome_distances = []
    for chrom in population:
        assignments = parse_chromosome(chrom, vehicles)
        vehicle_distances = calculate_vehicle_distances(
            assignments, customers, demands, vehicles, coordinates
        )
        total_distance = sum(vehicle_distances[v] for v in vehicles)
        chromosome_distances.append({
            'chromosome': chrom,
            'assignments': assignments,
            'vehicle_distances': vehicle_distances,
            'total_distance': total_distance
        })

    chromosome_distances.sort(key=lambda x: x['total_distance'])

    print("\nRanked Chromosomes by Total Distance Travelled:")
    for rank, chrom_info in enumerate(chromosome_distances, 1):
        print(f"\nRank {rank}:")
        print(f"  Chromosome: {chrom_info['chromosome']}")
        for v in vehicles:
            print(f"    Vehicle {v} serves: {chrom_info['assignments'][v]}")
        print(f"  Total Distance: {chrom_info['total_distance']:.2f}")
        for v in vehicles:
            print(f"    Vehicle {v} distance: {chrom_info['vehicle_distances'][v]:.2f}")

    print("\nCustomer Coordinates:")
    for i, coord in enumerate(coordinates, 1):
        print(f"Customer {i}: {coord}")

if __name__ == "__main__":
    genetic_algorithm()


Enter the number of customers:  4
Enter the number of vehicles:  3
Enter demand for customer 1:  16
Enter x-coordinate for customer 1:  23
Enter y-coordinate for customer 1:  -41
Enter demand for customer 2:  19
Enter x-coordinate for customer 2:  -43
Enter y-coordinate for customer 2:  -12
Enter demand for customer 3:  12
Enter x-coordinate for customer 3:  34
Enter y-coordinate for customer 3:  24
Enter demand for customer 4:  14
Enter x-coordinate for customer 4:  30
Enter y-coordinate for customer 4:  9
Enter the load carrying capacity of each truck:  28
Enter the population size:  10
Enter the number of generations:  40



Ranked Chromosomes by Total Distance Travelled:

Rank 1:
  Chromosome: ['1', '5', '3', '4', '7', '2', '6']
    Vehicle 5 serves: ['1']
    Vehicle 6 serves: ['2']
    Vehicle 7 serves: ['3', '4']
  Total Distance: 271.77
    Vehicle 5 distance: 94.02
    Vehicle 6 distance: 89.29
    Vehicle 7 distance: 88.46

Rank 2:
  Chromosome: ['3', '4', '5', '2', '6', '1', '7']
    Vehicle 5 serves: ['3', '4']
    Vehicle 6 serves: ['2']
    Vehicle 7 serves: ['1']
  Total Distance: 271.77
    Vehicle 5 distance: 88.46
    Vehicle 6 distance: 89.29
    Vehicle 7 distance: 94.02

Rank 3:
  Chromosome: ['3', '4', '6', '2', '5', '1', '7']
    Vehicle 5 serves: ['2']
    Vehicle 6 serves: ['3', '4']
    Vehicle 7 serves: ['1']
  Total Distance: 271.77
    Vehicle 5 distance: 89.29
    Vehicle 6 distance: 88.46
    Vehicle 7 distance: 94.02

Rank 4:
  Chromosome: ['3', '4', '5', '2', '6', '1', '7']
    Vehicle 5 serves: ['3', '4']
    Vehicle 6 serves: ['2']
    Vehicle 7 serves: ['1']
  Total Distan

In [1]:
import random
import math

def split_customer_demand(customer_id, demand, capacity):
    splits = []
    while demand > capacity:
        splits.append((f"{customer_id}p{len(splits)+1}", capacity))
        demand -= capacity
    if demand > 0:
        splits.append((f"{customer_id}p{len(splits)+1}", demand))
    return splits

def get_customer_parts(customers, demands, capacity):
    customer_parts = []
    demand_parts = []
    for c, d in zip(customers, demands):
        if d > capacity:
            splits = split_customer_demand(c, d, capacity)
            for label, part_demand in splits:
                customer_parts.append(label)
                demand_parts.append(part_demand)
        else:
            customer_parts.append(str(c))
            demand_parts.append(d)
    return customer_parts, demand_parts

def build_demands_dict(customers, demands, capacity):
    demands_dict = {}
    for c, d in zip(customers, demands):
        if d > capacity:
            splits = split_customer_demand(c, d, capacity)
            for label, part_demand in splits:
                demands_dict[label] = part_demand
        else:
            demands_dict[str(c)] = d
    return demands_dict

def generate_random_chromosome(customer_parts, demand_parts, vehicles, capacity):
    unassigned = list(zip(customer_parts, demand_parts))
    random.shuffle(unassigned)
    vehicles_left = vehicles[:]
    random.shuffle(vehicles_left)
    chromosome = []
    used_vehicles = set()
    i = 0
    n = len(unassigned)
    while i < n and vehicles_left:
        v = vehicles_left.pop()
        used_vehicles.add(v)
        group = []
        load = 0
        while i < n and load + unassigned[i][1] <= capacity:
            group.append(unassigned[i][0])
            load += unassigned[i][1]
            i += 1
        if group:
            chromosome.extend(group)
            chromosome.append(str(v))
    if i < n:
        return None
    for v in vehicles:
        if v not in used_vehicles:
            chromosome.append(str(v))
    return chromosome

def parse_chromosome(chromosome, vehicles):
    vehicle_set = set(str(v) for v in vehicles)
    assignments = {v: [] for v in vehicles}
    i = 0
    n = len(chromosome)
    while i < n:
        group = []
        while i < n and chromosome[i] not in vehicle_set:
            group.append(chromosome[i])
            i += 1
        if i < n and chromosome[i] in vehicle_set:
            v = int(chromosome[i])
            assignments[v].extend(group)
            i += 1
    return assignments

def is_legitimate_chromosome(chrom, customer_parts, vehicles, demands_dict, capacity):
    assigned_customers = set()
    assignments = parse_chromosome(chrom, vehicles)
    for v, cust_list in assignments.items():
        total_load = 0
        for cust in cust_list:
            if cust not in customer_parts or cust in assigned_customers:
                return False
            assigned_customers.add(cust)
            total_load += demands_dict[cust]
        if total_load > capacity:
            return False
    if assigned_customers != set(customer_parts):
        return False
    if len(chrom) != len(set(chrom)):
        return False
    if not all(str(v) in chrom for v in vehicles):
        return False
    return True

def calculate_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def calculate_vehicle_distances(assignments, customers, demands, vehicles, coordinates, depot=(0, 0)):
    cust_part_coords = {}
    for i, c in enumerate(customers):
        cust_part_coords[str(c)] = coordinates[i]
    for i, c in enumerate(customers):
        for part_num in range(2, 20):
            cust_part_coords[f"{c}p{part_num}"] = coordinates[i]
    vehicle_distances = {}
    for v, cust_list in assignments.items():
        if not cust_list:
            vehicle_distances[v] = 0.0
            continue
        total_dist = 0.0
        prev_point = depot
        for cust in cust_list:
            total_dist += calculate_distance(prev_point, cust_part_coords[cust])
            prev_point = cust_part_coords[cust]
        total_dist += calculate_distance(prev_point, depot)
        vehicle_distances[v] = total_dist
    return vehicle_distances

def single_point_crossover(parent1, parent2):
    length = len(parent1)
    if length < 2:
        return parent1[:], parent2[:]
    point = random.randint(1, length - 2)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def swap_mutation(chrom):
    c = chrom[:]
    idx1, idx2 = random.sample(range(len(c)), 2)
    c[idx1], c[idx2] = c[idx2], c[idx1]
    return c

def repair_chromosome(chrom, customer_parts, vehicles):
    seen = set()
    new_chrom = []
    for gene in chrom:
        if gene not in seen:
            new_chrom.append(gene)
            seen.add(gene)
    all_numbers = set(customer_parts + [str(v) for v in vehicles])
    missing = all_numbers - set(new_chrom)
    new_chrom += list(missing)
    return new_chrom[:len(customer_parts) + len(vehicles)]

def genetic_algorithm():
    # --- User Inputs ---
    num_customers = int(input("Enter the number of customers: "))
    num_vehicles = int(input("Enter the number of vehicles: "))
    customers = []
    demands = []
    coordinates = []
    for i in range(1, num_customers + 1):
        demand = int(input(f"Enter demand for customer {i}: "))
        x = float(input(f"Enter x-coordinate for customer {i}: "))
        y = float(input(f"Enter y-coordinate for customer {i}: "))
        customers.append(i)
        demands.append(demand)
        coordinates.append((x, y))
    vehicles = [i + num_customers for i in range(1, num_vehicles + 1)]
    truck_capacity = int(input("Enter the load carrying capacity of each truck: "))
    population_size = int(input("Enter the population size: "))
    num_generations = int(input("Enter the number of generations: "))

    # New: Get GA parameters from user
    crossover_prob = float(input("Enter crossover probability (e.g., 0.8): "))
    elitism_rate = float(input("Enter elitism rate (e.g., 0.9): "))
    mutation_prob = float(input("Enter mutation probability (e.g., 0.2): "))

    customer_parts, demand_parts = get_customer_parts(customers, demands, truck_capacity)
    demands_dict = build_demands_dict(customers, demands, truck_capacity)

    # Initialize population
    population = []
    attempts = 0
    while len(population) < population_size and attempts < population_size * 100:
        chrom = generate_random_chromosome(customer_parts, demand_parts, vehicles, truck_capacity)
        if chrom and is_legitimate_chromosome(chrom, customer_parts, vehicles, demands_dict, truck_capacity):
            population.append(chrom)
        attempts += 1

    if len(population) < population_size:
        print("Could not generate enough legitimate chromosomes for initial population.")
        return

    # Evolution
    for gen in range(num_generations):
        # Sort population by fitness (total distance) before elitism
        chromosome_distances = []
        for chrom in population:
            assignments = parse_chromosome(chrom, vehicles)
            vehicle_distances = calculate_vehicle_distances(
                assignments, customers, demands, vehicles, coordinates
            )
            total_distance = sum(vehicle_distances[v] for v in vehicles)
            chromosome_distances.append({
                'chromosome': chrom,
                'assignments': assignments,
                'vehicle_distances': vehicle_distances,
                'total_distance': total_distance
            })
        chromosome_distances.sort(key=lambda x: x['total_distance'])
        population = [cd['chromosome'] for cd in chromosome_distances]

        elitism_count = int(population_size * elitism_rate)
        next_population = population[:elitism_count]
        while len(next_population) < population_size:
            if random.random() < crossover_prob:  # Crossover probability
                parent1, parent2 = random.sample(population, 2)
                child1, child2 = single_point_crossover(parent1, parent2)
            else:
                # No crossover, just copy parents
                child1, child2 = random.sample(population, 2)
            if random.random() < mutation_prob:
                child1 = swap_mutation(child1)
            if random.random() < mutation_prob:
                child2 = swap_mutation(child2)
            child1 = repair_chromosome(child1, customer_parts, vehicles)
            child2 = repair_chromosome(child2, customer_parts, vehicles)
            if is_legitimate_chromosome(child1, customer_parts, vehicles, demands_dict, truck_capacity):
                next_population.append(child1)
            else:
                new_chrom = None
                attempts = 0
                while not new_chrom and attempts < 100:
                    new_chrom = generate_random_chromosome(customer_parts, demand_parts, vehicles, truck_capacity)
                    if new_chrom and not is_legitimate_chromosome(new_chrom, customer_parts, vehicles, demands_dict, truck_capacity):
                        new_chrom = None
                    attempts += 1
                if new_chrom:
                    next_population.append(new_chrom)
            if len(next_population) < population_size:
                if is_legitimate_chromosome(child2, customer_parts, vehicles, demands_dict, truck_capacity):
                    next_population.append(child2)
                else:
                    new_chrom = None
                    attempts = 0
                    while not new_chrom and attempts < 100:
                        new_chrom = generate_random_chromosome(customer_parts, demand_parts, vehicles, truck_capacity)
                        if new_chrom and not is_legitimate_chromosome(new_chrom, customer_parts, vehicles, demands_dict, truck_capacity):
                            new_chrom = None
                        attempts += 1
                    if new_chrom:
                        next_population.append(new_chrom)
        population = next_population[:population_size]

    # Evaluate and rank final population
    chromosome_distances = []
    for chrom in population:
        assignments = parse_chromosome(chrom, vehicles)
        vehicle_distances = calculate_vehicle_distances(
            assignments, customers, demands, vehicles, coordinates
        )
        total_distance = sum(vehicle_distances[v] for v in vehicles)
        chromosome_distances.append({
            'chromosome': chrom,
            'assignments': assignments,
            'vehicle_distances': vehicle_distances,
            'total_distance': total_distance
        })

    chromosome_distances.sort(key=lambda x: x['total_distance'])

    print("\nRanked Chromosomes by Total Distance Travelled:")
    for rank, chrom_info in enumerate(chromosome_distances, 1):
        print(f"\nRank {rank}:")
        print(f"  Chromosome: {chrom_info['chromosome']}")
        for v in vehicles:
            print(f"    Vehicle {v} serves: {chrom_info['assignments'][v]}")
        print(f"  Total Distance: {chrom_info['total_distance']:.2f}")
        for v in vehicles:
            print(f"    Vehicle {v} distance: {chrom_info['vehicle_distances'][v]:.2f}")

    print("\nCustomer Coordinates:")
    for i, coord in enumerate(coordinates, 1):
        print(f"Customer {i}: {coord}")

if __name__ == "__main__":
    genetic_algorithm()


Enter the number of customers:  4
Enter the number of vehicles:  3
Enter demand for customer 1:  19
Enter x-coordinate for customer 1:  23
Enter y-coordinate for customer 1:  -42
Enter demand for customer 2:  16
Enter x-coordinate for customer 2:  -33
Enter y-coordinate for customer 2:  -12
Enter demand for customer 3:  12
Enter x-coordinate for customer 3:  32
Enter y-coordinate for customer 3:  45
Enter demand for customer 4:  14
Enter x-coordinate for customer 4:  10
Enter y-coordinate for customer 4:  7
Enter the load carrying capacity of each truck:  32
Enter the population size:  10
Enter the number of generations:  20
Enter crossover probability (e.g., 0.8):  0.8
Enter elitism rate (e.g., 0.9):  0.9
Enter mutation probability (e.g., 0.2):  0.2



Ranked Chromosomes by Total Distance Travelled:

Rank 1:
  Chromosome: ['3', '4', '7', '1', '6', '2', '5']
    Vehicle 5 serves: ['2']
    Vehicle 6 serves: ['1']
    Vehicle 7 serves: ['3', '4']
  Total Distance: 277.33
    Vehicle 5 distance: 70.23
    Vehicle 6 distance: 95.77
    Vehicle 7 distance: 111.33

Rank 2:
  Chromosome: ['3', '4', '6', '2', '7', '1', '5']
    Vehicle 5 serves: ['1']
    Vehicle 6 serves: ['3', '4']
    Vehicle 7 serves: ['2']
  Total Distance: 277.33
    Vehicle 5 distance: 95.77
    Vehicle 6 distance: 111.33
    Vehicle 7 distance: 70.23

Rank 3:
  Chromosome: ['4', '3', '5', '1', '7', '2', '6']
    Vehicle 5 serves: ['4', '3']
    Vehicle 6 serves: ['2']
    Vehicle 7 serves: ['1']
  Total Distance: 277.33
    Vehicle 5 distance: 111.33
    Vehicle 6 distance: 70.23
    Vehicle 7 distance: 95.77

Rank 4:
  Chromosome: ['2', '7', '1', '6', '4', '3', '5']
    Vehicle 5 serves: ['4', '3']
    Vehicle 6 serves: ['1']
    Vehicle 7 serves: ['2']
  Total Dis

In [1]:
import random
import math

def split_customer_demand(customer_id, demand, capacity):
    splits = []
    while demand > capacity:
        splits.append((f"{customer_id}p{len(splits)+1}", capacity))
        demand -= capacity
    if demand > 0:
        splits.append((f"{customer_id}p{len(splits)+1}", demand))
    return splits

def get_customer_parts(customers, demands, capacity):
    customer_parts = []
    demand_parts = []
    for c, d in zip(customers, demands):
        if d > capacity:
            splits = split_customer_demand(c, d, capacity)
            for label, part_demand in splits:
                customer_parts.append(label)
                demand_parts.append(part_demand)
        else:
            customer_parts.append(str(c))
            demand_parts.append(d)
    return customer_parts, demand_parts

def build_demands_dict(customers, demands, capacity):
    demands_dict = {}
    for c, d in zip(customers, demands):
        if d > capacity:
            splits = split_customer_demand(c, d, capacity)
            for label, part_demand in splits:
                demands_dict[label] = part_demand
        else:
            demands_dict[str(c)] = d
    return demands_dict

def generate_random_chromosome(customer_parts, demand_parts, vehicles, capacity):
    unassigned = list(zip(customer_parts, demand_parts))
    random.shuffle(unassigned)
    vehicles_left = vehicles[:]
    random.shuffle(vehicles_left)
    chromosome = []
    used_vehicles = set()
    i = 0
    n = len(unassigned)
    while i < n and vehicles_left:
        v = vehicles_left.pop()
        used_vehicles.add(v)
        group = []
        load = 0
        while i < n and load + unassigned[i][1] <= capacity:
            group.append(unassigned[i][0])
            load += unassigned[i][1]
            i += 1
        if group:
            chromosome.extend(group)
            chromosome.append(str(v))
    if i < n:
        return None
    for v in vehicles:
        if v not in used_vehicles:
            chromosome.append(str(v))
    return chromosome

def parse_chromosome(chromosome, vehicles):
    vehicle_set = set(str(v) for v in vehicles)
    assignments = {v: [] for v in vehicles}
    i = 0
    n = len(chromosome)
    while i < n:
        group = []
        while i < n and chromosome[i] not in vehicle_set:
            group.append(chromosome[i])
            i += 1
        if i < n and chromosome[i] in vehicle_set:
            v = int(chromosome[i])
            assignments[v].extend(group)
            i += 1
    return assignments

def is_legitimate_chromosome(chrom, customer_parts, vehicles, demands_dict, capacity):
    assigned_customers = set()
    assignments = parse_chromosome(chrom, vehicles)
    for v, cust_list in assignments.items():
        total_load = 0
        for cust in cust_list:
            if cust not in customer_parts or cust in assigned_customers:
                return False
            assigned_customers.add(cust)
            total_load += demands_dict[cust]
        if total_load > capacity:
            return False
    if assigned_customers != set(customer_parts):
        return False
    if len(chrom) != len(set(chrom)):
        return False
    if not all(str(v) in chrom for v in vehicles):
        return False
    return True

def calculate_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def calculate_vehicle_distances(assignments, customers, demands, vehicles, coordinates, depot=(0, 0)):
    cust_part_coords = {}
    for i, c in enumerate(customers):
        cust_part_coords[str(c)] = coordinates[i]
    for i, c in enumerate(customers):
        for part_num in range(2, 20):
            cust_part_coords[f"{c}p{part_num}"] = coordinates[i]
    vehicle_distances = {}
    for v, cust_list in assignments.items():
        if not cust_list:
            vehicle_distances[v] = 0.0
            continue
        total_dist = 0.0
        prev_point = depot
        for cust in cust_list:
            total_dist += calculate_distance(prev_point, cust_part_coords[cust])
            prev_point = cust_part_coords[cust]
        total_dist += calculate_distance(prev_point, depot)
        vehicle_distances[v] = total_dist
    return vehicle_distances

def single_point_crossover(parent1, parent2):
    length = len(parent1)
    if length < 2:
        return parent1[:], parent2[:]
    point = random.randint(1, length - 2)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def swap_mutation(chrom):
    c = chrom[:]
    idx1, idx2 = random.sample(range(len(c)), 2)
    c[idx1], c[idx2] = c[idx2], c[idx1]
    return c

def repair_chromosome(chrom, customer_parts, vehicles):
    seen = set()
    new_chrom = []
    for gene in chrom:
        if gene not in seen:
            new_chrom.append(gene)
            seen.add(gene)
    all_numbers = set(customer_parts + [str(v) for v in vehicles])
    missing = all_numbers - set(new_chrom)
    new_chrom += list(missing)
    return new_chrom[:len(customer_parts) + len(vehicles)]

def fitness_function(total_distance):
    if total_distance == 0:
        return float('inf')  # Best possible
    return 1 / total_distance

def genetic_algorithm():
    # --- User Inputs ---
    num_customers = int(input("Enter the number of customers: "))
    num_vehicles = int(input("Enter the number of vehicles: "))
    customers = []
    demands = []
    coordinates = []
    for i in range(1, num_customers + 1):
        demand = int(input(f"Enter demand for customer {i}: "))
        x = float(input(f"Enter x-coordinate for customer {i}: "))
        y = float(input(f"Enter y-coordinate for customer {i}: "))
        customers.append(i)
        demands.append(demand)
        coordinates.append((x, y))
    vehicles = [i + num_customers for i in range(1, num_vehicles + 1)]
    truck_capacity = int(input("Enter the load carrying capacity of each truck: "))
    population_size = int(input("Enter the population size: "))
    num_generations = int(input("Enter the number of generations: "))

    # New: Get GA parameters from user
    crossover_prob = float(input("Enter crossover probability (e.g., 0.8): "))
    elitism_rate = float(input("Enter elitism rate (e.g., 0.9): "))
    mutation_prob = float(input("Enter mutation probability (e.g., 0.2): "))

    customer_parts, demand_parts = get_customer_parts(customers, demands, truck_capacity)
    demands_dict = build_demands_dict(customers, demands, truck_capacity)

    # Initialize population
    population = []
    attempts = 0
    while len(population) < population_size and attempts < population_size * 100:
        chrom = generate_random_chromosome(customer_parts, demand_parts, vehicles, truck_capacity)
        if chrom and is_legitimate_chromosome(chrom, customer_parts, vehicles, demands_dict, truck_capacity):
            population.append(chrom)
        attempts += 1

    if len(population) < population_size:
        print("Could not generate enough legitimate chromosomes for initial population.")
        return

    # Evolution
    for gen in range(num_generations):
        # Sort population by fitness (reciprocal of total distance) before elitism
        chromosome_fitness = []
        for chrom in population:
            assignments = parse_chromosome(chrom, vehicles)
            vehicle_distances = calculate_vehicle_distances(
                assignments, customers, demands, vehicles, coordinates
            )
            total_distance = sum(vehicle_distances[v] for v in vehicles)
            fitness = fitness_function(total_distance)
            chromosome_fitness.append({
                'chromosome': chrom,
                'assignments': assignments,
                'vehicle_distances': vehicle_distances,
                'total_distance': total_distance,
                'fitness': fitness
            })
        chromosome_fitness.sort(key=lambda x: x['fitness'], reverse=True)
        population = [cd['chromosome'] for cd in chromosome_fitness]

        elitism_count = int(population_size * elitism_rate)
        next_population = population[:elitism_count]
        while len(next_population) < population_size:
            if random.random() < crossover_prob:  # Crossover probability
                parent1, parent2 = random.sample(population, 2)
                child1, child2 = single_point_crossover(parent1, parent2)
            else:
                # No crossover, just copy parents
                child1, child2 = random.sample(population, 2)
            if random.random() < mutation_prob:
                child1 = swap_mutation(child1)
            if random.random() < mutation_prob:
                child2 = swap_mutation(child2)
            child1 = repair_chromosome(child1, customer_parts, vehicles)
            child2 = repair_chromosome(child2, customer_parts, vehicles)
            if is_legitimate_chromosome(child1, customer_parts, vehicles, demands_dict, truck_capacity):
                next_population.append(child1)
            else:
                new_chrom = None
                attempts = 0
                while not new_chrom and attempts < 100:
                    new_chrom = generate_random_chromosome(customer_parts, demand_parts, vehicles, truck_capacity)
                    if new_chrom and not is_legitimate_chromosome(new_chrom, customer_parts, vehicles, demands_dict, truck_capacity):
                        new_chrom = None
                    attempts += 1
                if new_chrom:
                    next_population.append(new_chrom)
            if len(next_population) < population_size:
                if is_legitimate_chromosome(child2, customer_parts, vehicles, demands_dict, truck_capacity):
                    next_population.append(child2)
                else:
                    new_chrom = None
                    attempts = 0
                    while not new_chrom and attempts < 100:
                        new_chrom = generate_random_chromosome(customer_parts, demand_parts, vehicles, truck_capacity)
                        if new_chrom and not is_legitimate_chromosome(new_chrom, customer_parts, vehicles, demands_dict, truck_capacity):
                            new_chrom = None
                        attempts += 1
                    if new_chrom:
                        next_population.append(new_chrom)
        population = next_population[:population_size]

    # Evaluate and rank final population by fitness (reciprocal of total distance)
    chromosome_fitness = []
    for chrom in population:
        assignments = parse_chromosome(chrom, vehicles)
        vehicle_distances = calculate_vehicle_distances(
            assignments, customers, demands, vehicles, coordinates
        )
        total_distance = sum(vehicle_distances[v] for v in vehicles)
        fitness = fitness_function(total_distance)
        chromosome_fitness.append({
            'chromosome': chrom,
            'assignments': assignments,
            'vehicle_distances': vehicle_distances,
            'total_distance': total_distance,
            'fitness': fitness
        })

    chromosome_fitness.sort(key=lambda x: x['fitness'], reverse=True)

    print("\nRanked Chromosomes by Fitness (Reciprocal of Total Distance):")
    for rank, chrom_info in enumerate(chromosome_fitness, 1):
        print(f"\nRank {rank}:")
        print(f"  Chromosome: {chrom_info['chromosome']}")
        for v in vehicles:
            print(f"    Vehicle {v} serves: {chrom_info['assignments'][v]}")
        print(f"  Total Distance: {chrom_info['total_distance']:.2f}")
        print(f"  Fitness Value: {chrom_info['fitness']:.6f}")
        for v in vehicles:
            print(f"    Vehicle {v} distance: {chrom_info['vehicle_distances'][v]:.2f}")

    print("\nCustomer Coordinates:")
    for i, coord in enumerate(coordinates, 1):
        print(f"Customer {i}: {coord}")

if __name__ == "__main__":
    genetic_algorithm()


Enter the number of customers:  4
Enter the number of vehicles:  3
Enter demand for customer 1:  16
Enter x-coordinate for customer 1:  23
Enter y-coordinate for customer 1:  -41
Enter demand for customer 2:  19
Enter x-coordinate for customer 2:  -43
Enter y-coordinate for customer 2:  -12
Enter demand for customer 3:  12
Enter x-coordinate for customer 3:  34
Enter y-coordinate for customer 3:  24
Enter demand for customer 4:  14
Enter x-coordinate for customer 4:  30
Enter y-coordinate for customer 4:  9
Enter the load carrying capacity of each truck:  28
Enter the population size:  10
Enter the number of generations:  20
Enter crossover probability (e.g., 0.8):  0.8
Enter elitism rate (e.g., 0.9):  0.1
Enter mutation probability (e.g., 0.2):  0.2



Ranked Chromosomes by Fitness (Reciprocal of Total Distance):

Rank 1:
  Chromosome: ['3', '4', '6', '2', '5', '1', '7']
    Vehicle 5 serves: ['2']
    Vehicle 6 serves: ['3', '4']
    Vehicle 7 serves: ['1']
  Total Distance: 271.77
  Fitness Value: 0.003680
    Vehicle 5 distance: 89.29
    Vehicle 6 distance: 88.46
    Vehicle 7 distance: 94.02

Rank 2:
  Chromosome: ['4', '3', '5', '2', '6', '1', '7']
    Vehicle 5 serves: ['4', '3']
    Vehicle 6 serves: ['2']
    Vehicle 7 serves: ['1']
  Total Distance: 271.77
  Fitness Value: 0.003680
    Vehicle 5 distance: 88.46
    Vehicle 6 distance: 89.29
    Vehicle 7 distance: 94.02

Rank 3:
  Chromosome: ['2', '5', '4', '3', '6', '1', '7']
    Vehicle 5 serves: ['2']
    Vehicle 6 serves: ['4', '3']
    Vehicle 7 serves: ['1']
  Total Distance: 271.77
  Fitness Value: 0.003680
    Vehicle 5 distance: 89.29
    Vehicle 6 distance: 88.46
    Vehicle 7 distance: 94.02

Rank 4:
  Chromosome: ['3', '4', '6', '2', '5', '1', '7']
    Vehicle

In [2]:
import random
import math

def split_customer_demand(customer_id, demand, capacity):
    splits = []
    while demand > capacity:
        splits.append((f"{customer_id}p{len(splits)+1}", capacity))
        demand -= capacity
    if demand > 0:
        splits.append((f"{customer_id}p{len(splits)+1}", demand))
    return splits

def get_customer_parts(customers, demands, capacity):
    customer_parts = []
    demand_parts = []
    for c, d in zip(customers, demands):
        if d > capacity:
            splits = split_customer_demand(c, d, capacity)
            for label, part_demand in splits:
                customer_parts.append(label)
                demand_parts.append(part_demand)
        else:
            customer_parts.append(str(c))
            demand_parts.append(d)
    return customer_parts, demand_parts

def build_demands_dict(customers, demands, capacity):
    demands_dict = {}
    for c, d in zip(customers, demands):
        if d > capacity:
            splits = split_customer_demand(c, d, capacity)
            for label, part_demand in splits:
                demands_dict[label] = part_demand
        else:
            demands_dict[str(c)] = d
    return demands_dict

def generate_random_chromosome(customer_parts, demand_parts, vehicles, capacity):
    unassigned = list(zip(customer_parts, demand_parts))
    random.shuffle(unassigned)
    vehicles_left = vehicles[:]
    random.shuffle(vehicles_left)
    chromosome = []
    used_vehicles = set()
    i = 0
    n = len(unassigned)
    while i < n and vehicles_left:
        v = vehicles_left.pop()
        used_vehicles.add(v)
        group = []
        load = 0
        while i < n and load + unassigned[i][1] <= capacity:
            group.append(unassigned[i][0])
            load += unassigned[i][1]
            i += 1
        if group:
            chromosome.extend(group)
            chromosome.append(str(v))
    if i < n:
        return None
    for v in vehicles:
        if v not in used_vehicles:
            chromosome.append(str(v))
    return chromosome

def parse_chromosome(chromosome, vehicles):
    vehicle_set = set(str(v) for v in vehicles)
    assignments = {v: [] for v in vehicles}
    i = 0
    n = len(chromosome)
    while i < n:
        group = []
        while i < n and chromosome[i] not in vehicle_set:
            group.append(chromosome[i])
            i += 1
        if i < n and chromosome[i] in vehicle_set:
            v = int(chromosome[i])
            assignments[v].extend(group)
            i += 1
    return assignments

def is_legitimate_chromosome(chrom, customer_parts, vehicles, demands_dict, capacity):
    assigned_customers = set()
    assignments = parse_chromosome(chrom, vehicles)
    for v, cust_list in assignments.items():
        total_load = 0
        for cust in cust_list:
            if cust not in customer_parts or cust in assigned_customers:
                return False
            assigned_customers.add(cust)
            total_load += demands_dict[cust]
        if total_load > capacity:
            return False
    if assigned_customers != set(customer_parts):
        return False
    if len(chrom) != len(set(chrom)):
        return False
    if not all(str(v) in chrom for v in vehicles):
        return False
    return True

def calculate_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def calculate_vehicle_distances(assignments, customers, demands, vehicles, coordinates, depot=(0, 0)):
    cust_part_coords = {}
    for i, c in enumerate(customers):
        cust_part_coords[str(c)] = coordinates[i]
    for i, c in enumerate(customers):
        for part_num in range(2, 20):
            cust_part_coords[f"{c}p{part_num}"] = coordinates[i]
    vehicle_distances = {}
    for v, cust_list in assignments.items():
        if not cust_list:
            vehicle_distances[v] = 0.0
            continue
        total_dist = 0.0
        prev_point = depot
        for cust in cust_list:
            total_dist += calculate_distance(prev_point, cust_part_coords[cust])
            prev_point = cust_part_coords[cust]
        total_dist += calculate_distance(prev_point, depot)
        vehicle_distances[v] = total_dist
    return vehicle_distances

def single_point_crossover(parent1, parent2):
    length = len(parent1)
    if length < 2:
        return parent1[:], parent2[:]
    point = random.randint(1, length - 2)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def swap_mutation(chrom):
    c = chrom[:]
    idx1, idx2 = random.sample(range(len(c)), 2)
    c[idx1], c[idx2] = c[idx2], c[idx1]
    return c

def repair_chromosome(chrom, customer_parts, vehicles):
    seen = set()
    new_chrom = []
    for gene in chrom:
        if gene not in seen:
            new_chrom.append(gene)
            seen.add(gene)
    all_numbers = set(customer_parts + [str(v) for v in vehicles])
    missing = all_numbers - set(new_chrom)
    new_chrom += list(missing)
    return new_chrom[:len(customer_parts) + len(vehicles)]

def fitness_function(total_distance):
    if total_distance == 0:
        return float('inf')  # Best possible
    return 1 / total_distance

def genetic_algorithm():
    # --- User Inputs ---
    num_customers = int(input("Enter the number of customers: "))
    num_vehicles = int(input("Enter the number of vehicles: "))
    customers = []
    demands = []
    coordinates = []
    for i in range(1, num_customers + 1):
        demand = int(input(f"Enter demand for customer {i}: "))
        x = float(input(f"Enter x-coordinate for customer {i}: "))
        y = float(input(f"Enter y-coordinate for customer {i}: "))
        customers.append(i)
        demands.append(demand)
        coordinates.append((x, y))
    vehicles = [i + num_customers for i in range(1, num_vehicles + 1)]
    truck_capacity = int(input("Enter the load carrying capacity of each truck: "))
    population_size = int(input("Enter the population size: "))
    num_generations = int(input("Enter the number of generations: "))

    crossover_prob = float(input("Enter crossover probability (e.g., 0.8): "))
    elitism_rate = float(input("Enter elitism rate (e.g., 0.9): "))
    mutation_prob = float(input("Enter mutation probability (e.g., 0.2): "))

    customer_parts, demand_parts = get_customer_parts(customers, demands, truck_capacity)
    demands_dict = build_demands_dict(customers, demands, truck_capacity)

    # Initialize population
    population = []
    attempts = 0
    while len(population) < population_size and attempts < population_size * 100:
        chrom = generate_random_chromosome(customer_parts, demand_parts, vehicles, truck_capacity)
        if chrom and is_legitimate_chromosome(chrom, customer_parts, vehicles, demands_dict, truck_capacity):
            if chrom not in population:
                population.append(chrom)
        attempts += 1

    if len(population) < population_size:
        print("Could not generate enough legitimate chromosomes for initial population.")
        return

    # Evolution
    for gen in range(num_generations):
        chromosome_fitness = []
        for chrom in population:
            assignments = parse_chromosome(chrom, vehicles)
            vehicle_distances = calculate_vehicle_distances(
                assignments, customers, demands, vehicles, coordinates
            )
            total_distance = sum(vehicle_distances[v] for v in vehicles)
            fitness = fitness_function(total_distance)
            chromosome_fitness.append({
                'chromosome': chrom,
                'assignments': assignments,
                'vehicle_distances': vehicle_distances,
                'total_distance': total_distance,
                'fitness': fitness
            })
        chromosome_fitness.sort(key=lambda x: x['fitness'], reverse=True)
        population = [cd['chromosome'] for cd in chromosome_fitness]

        elitism_count = int(population_size * elitism_rate)
        next_population = population[:elitism_count]
        prev_generation = population[:]

        # Generate rest of the population
        while len(next_population) < population_size:
            if random.random() < crossover_prob:
                parent1, parent2 = random.sample(population, 2)
                child1, child2 = single_point_crossover(parent1, parent2)
            else:
                child1, child2 = random.sample(population, 2)
            if random.random() < mutation_prob:
                child1 = swap_mutation(child1)
            if random.random() < mutation_prob:
                child2 = swap_mutation(child2)
            child1 = repair_chromosome(child1, customer_parts, vehicles)
            child2 = repair_chromosome(child2, customer_parts, vehicles)

            # Add child1 if valid and not duplicate
            if is_legitimate_chromosome(child1, customer_parts, vehicles, demands_dict, truck_capacity):
                if child1 not in next_population:
                    next_population.append(child1)
            # Add child2 if valid and not duplicate and space left
            if len(next_population) < population_size:
                if is_legitimate_chromosome(child2, customer_parts, vehicles, demands_dict, truck_capacity):
                    if child2 not in next_population:
                        next_population.append(child2)

        # Duplicate prevention: Replace any duplicates (except elitism) with valid chromosomes from previous generation
        seen = set(tuple(chrom) for chrom in next_population[:elitism_count])
        unique_population = next_population[:elitism_count]
        # Start checking from the end of elitism
        i = elitism_count
        while i < len(next_population):
            chrom_tuple = tuple(next_population[i])
            if chrom_tuple not in seen:
                unique_population.append(next_population[i])
                seen.add(chrom_tuple)
            else:
                # Replace duplicate with a valid, non-duplicate chromosome from previous generation (not in elitism)
                replaced = False
                for candidate in prev_generation[elitism_count:]:
                    cand_tuple = tuple(candidate)
                    if cand_tuple not in seen:
                        unique_population.append(candidate)
                        seen.add(cand_tuple)
                        replaced = True
                        break
                # If no replacement found, just skip (should not happen if population_size is reasonable)
            i += 1
        # If after replacement we have less than needed, fill with random valid chromosomes from prev generation (not elitism)
        while len(unique_population) < population_size:
            candidate = random.choice(prev_generation[elitism_count:])
            cand_tuple = tuple(candidate)
            if cand_tuple not in seen:
                unique_population.append(candidate)
                seen.add(cand_tuple)
        population = unique_population[:population_size]

    # Evaluate and rank final population by fitness (reciprocal of total distance)
    chromosome_fitness = []
    for chrom in population:
        assignments = parse_chromosome(chrom, vehicles)
        vehicle_distances = calculate_vehicle_distances(
            assignments, customers, demands, vehicles, coordinates
        )
        total_distance = sum(vehicle_distances[v] for v in vehicles)
        fitness = fitness_function(total_distance)
        chromosome_fitness.append({
            'chromosome': chrom,
            'assignments': assignments,
            'vehicle_distances': vehicle_distances,
            'total_distance': total_distance,
            'fitness': fitness
        })

    chromosome_fitness.sort(key=lambda x: x['fitness'], reverse=True)

    print("\nRanked Chromosomes by Fitness (Reciprocal of Total Distance):")
    for rank, chrom_info in enumerate(chromosome_fitness, 1):
        print(f"\nRank {rank}:")
        print(f"  Chromosome: {chrom_info['chromosome']}")
        for v in vehicles:
            print(f"    Vehicle {v} serves: {chrom_info['assignments'][v]}")
        print(f"  Total Distance: {chrom_info['total_distance']:.2f}")
        print(f"  Fitness Value: {chrom_info['fitness']:.6f}")
        for v in vehicles:
            print(f"    Vehicle {v} distance: {chrom_info['vehicle_distances'][v]:.2f}")

    print("\nCustomer Coordinates:")
    for i, coord in enumerate(coordinates, 1):
        print(f"Customer {i}: {coord}")

if __name__ == "__main__":
    genetic_algorithm()


Enter the number of customers:  4
Enter the number of vehicles:  3
Enter demand for customer 1:  16
Enter x-coordinate for customer 1:  23
Enter y-coordinate for customer 1:  -41
Enter demand for customer 2:  19
Enter x-coordinate for customer 2:  -43
Enter y-coordinate for customer 2:  -12
Enter demand for customer 3:  12
Enter x-coordinate for customer 3:  34
Enter y-coordinate for customer 3:  24
Enter demand for customer 4:  14
Enter x-coordinate for customer 4:  30
Enter y-coordinate for customer 4:  9
Enter the load carrying capacity of each truck:  28
Enter the population size:  10
Enter the number of generations:  20
Enter crossover probability (e.g., 0.8):  0.8
Enter elitism rate (e.g., 0.9):  0.1
Enter mutation probability (e.g., 0.2):  0.2



Ranked Chromosomes by Fitness (Reciprocal of Total Distance):

Rank 1:
  Chromosome: ['1', '6', '2', '5', '4', '3', '7']
    Vehicle 5 serves: ['2']
    Vehicle 6 serves: ['1']
    Vehicle 7 serves: ['4', '3']
  Total Distance: 271.77
  Fitness Value: 0.003680
    Vehicle 5 distance: 89.29
    Vehicle 6 distance: 94.02
    Vehicle 7 distance: 88.46

Rank 2:
  Chromosome: ['1', '6', '2', '7', '4', '3', '5']
    Vehicle 5 serves: ['4', '3']
    Vehicle 6 serves: ['1']
    Vehicle 7 serves: ['2']
  Total Distance: 271.77
  Fitness Value: 0.003680
    Vehicle 5 distance: 88.46
    Vehicle 6 distance: 94.02
    Vehicle 7 distance: 89.29

Rank 3:
  Chromosome: ['1', '6', '3', '4', '7', '2', '5']
    Vehicle 5 serves: ['2']
    Vehicle 6 serves: ['1']
    Vehicle 7 serves: ['3', '4']
  Total Distance: 271.77
  Fitness Value: 0.003680
    Vehicle 5 distance: 89.29
    Vehicle 6 distance: 94.02
    Vehicle 7 distance: 88.46

Rank 4:
  Chromosome: ['1', '3', '6', '4', '7', '2', '5']
    Vehicle

In [3]:
import random
import math

def split_customer_demand(customer_id, demand, capacity):
    splits = []
    while demand > capacity:
        splits.append((f"{customer_id}p{len(splits)+1}", capacity))
        demand -= capacity
    if demand > 0:
        splits.append((f"{customer_id}p{len(splits)+1}", demand))
    return splits

def get_customer_parts(customers, demands, capacity):
    customer_parts = []
    demand_parts = []
    for c, d in zip(customers, demands):
        if d > capacity:
            splits = split_customer_demand(c, d, capacity)
            for label, part_demand in splits:
                customer_parts.append(label)
                demand_parts.append(part_demand)
        else:
            customer_parts.append(str(c))
            demand_parts.append(d)
    return customer_parts, demand_parts

def build_demands_dict(customers, demands, capacity):
    demands_dict = {}
    for c, d in zip(customers, demands):
        if d > capacity:
            splits = split_customer_demand(c, d, capacity)
            for label, part_demand in splits:
                demands_dict[label] = part_demand
        else:
            demands_dict[str(c)] = d
    return demands_dict

def generate_random_chromosome(customer_parts, demand_parts, vehicles, capacity):
    unassigned = list(zip(customer_parts, demand_parts))
    random.shuffle(unassigned)
    vehicles_left = vehicles[:]
    random.shuffle(vehicles_left)
    chromosome = []
    used_vehicles = set()
    i = 0
    n = len(unassigned)
    while i < n and vehicles_left:
        v = vehicles_left.pop()
        used_vehicles.add(v)
        group = []
        load = 0
        while i < n and load + unassigned[i][1] <= capacity:
            group.append(unassigned[i][0])
            load += unassigned[i][1]
            i += 1
        if group:
            chromosome.extend(group)
            chromosome.append(str(v))
    if i < n:
        return None
    for v in vehicles:
        if v not in used_vehicles:
            chromosome.append(str(v))
    return chromosome

def parse_chromosome(chromosome, vehicles):
    vehicle_set = set(str(v) for v in vehicles)
    assignments = {v: [] for v in vehicles}
    i = 0
    n = len(chromosome)
    while i < n:
        group = []
        while i < n and chromosome[i] not in vehicle_set:
            group.append(chromosome[i])
            i += 1
        if i < n and chromosome[i] in vehicle_set:
            v = int(chromosome[i])
            assignments[v].extend(group)
            i += 1
    return assignments

def is_legitimate_chromosome(chrom, customer_parts, vehicles, demands_dict, capacity):
    assigned_customers = set()
    assignments = parse_chromosome(chrom, vehicles)
    for v, cust_list in assignments.items():
        total_load = 0
        for cust in cust_list:
            if cust not in customer_parts or cust in assigned_customers:
                return False
            assigned_customers.add(cust)
            total_load += demands_dict[cust]
        if total_load > capacity:
            return False
    if assigned_customers != set(customer_parts):
        return False
    if len(chrom) != len(set(chrom)):
        return False
    if not all(str(v) in chrom for v in vehicles):
        return False
    return True

def calculate_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def calculate_vehicle_distances(assignments, customers, demands, vehicles, coordinates, depot=(0, 0)):
    cust_part_coords = {}
    for i, c in enumerate(customers):
        cust_part_coords[str(c)] = coordinates[i]
    for i, c in enumerate(customers):
        for part_num in range(2, 20):
            cust_part_coords[f"{c}p{part_num}"] = coordinates[i]
    vehicle_distances = {}
    for v, cust_list in assignments.items():
        if not cust_list:
            vehicle_distances[v] = 0.0
            continue
        total_dist = 0.0
        prev_point = depot
        for cust in cust_list:
            total_dist += calculate_distance(prev_point, cust_part_coords[cust])
            prev_point = cust_part_coords[cust]
        total_dist += calculate_distance(prev_point, depot)
        vehicle_distances[v] = total_dist
    return vehicle_distances

def single_point_crossover(parent1, parent2):
    length = len(parent1)
    if length < 2:
        return parent1[:], parent2[:]
    point = random.randint(1, length - 2)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def swap_mutation(chrom):
    c = chrom[:]
    idx1, idx2 = random.sample(range(len(c)), 2)
    c[idx1], c[idx2] = c[idx2], c[idx1]
    return c

def repair_chromosome(chrom, customer_parts, vehicles):
    seen = set()
    new_chrom = []
    for gene in chrom:
        if gene not in seen:
            new_chrom.append(gene)
            seen.add(gene)
    all_numbers = set(customer_parts + [str(v) for v in vehicles])
    missing = all_numbers - set(new_chrom)
    new_chrom += list(missing)
    return new_chrom[:len(customer_parts) + len(vehicles)]

def fitness_function(total_distance):
    if total_distance == 0:
        return float('inf')  # Best possible
    return 1 / total_distance

def store_best_chromosomes(generation, chromosome_fitness, best_chromosomes):
    best = chromosome_fitness[0]
    best_chromosomes.append({
        'generation': generation,
        'chromosome': best['chromosome'],
        'fitness': best['fitness']
    })

def genetic_algorithm():
    num_customers = int(input("Enter the number of customers: "))
    num_vehicles = int(input("Enter the number of vehicles: "))
    customers = []
    demands = []
    coordinates = []
    for i in range(1, num_customers + 1):
        demand = int(input(f"Enter demand for customer {i}: "))
        x = float(input(f"Enter x-coordinate for customer {i}: "))
        y = float(input(f"Enter y-coordinate for customer {i}: "))
        customers.append(i)
        demands.append(demand)
        coordinates.append((x, y))
    vehicles = [i + num_customers for i in range(1, num_vehicles + 1)]
    truck_capacity = int(input("Enter the load carrying capacity of each truck: "))
    population_size = int(input("Enter the population size: "))
    max_unchanged = int(input("Enter the number of generations with unchanged fitness before stopping: "))

    crossover_prob = float(input("Enter crossover probability (e.g., 0.8): "))
    elitism_rate = float(input("Enter elitism rate (e.g., 0.9): "))
    mutation_prob = float(input("Enter mutation probability (e.g., 0.2): "))

    customer_parts, demand_parts = get_customer_parts(customers, demands, truck_capacity)
    demands_dict = build_demands_dict(customers, demands, truck_capacity)

    population = []
    attempts = 0
    while len(population) < population_size and attempts < population_size * 100:
        chrom = generate_random_chromosome(customer_parts, demand_parts, vehicles, truck_capacity)
        if chrom and is_legitimate_chromosome(chrom, customer_parts, vehicles, demands_dict, truck_capacity):
            if chrom not in population:
                population.append(chrom)
        attempts += 1

    if len(population) < population_size:
        print("Could not generate enough legitimate chromosomes for initial population.")
        return

    best_chromosomes = []
    unchanged_count = 0
    prev_best_fitness = None
    generation = 0

    while unchanged_count < max_unchanged:
        generation += 1
        chromosome_fitness = []
        for chrom in population:
            assignments = parse_chromosome(chrom, vehicles)
            vehicle_distances = calculate_vehicle_distances(
                assignments, customers, demands, vehicles, coordinates
            )
            total_distance = sum(vehicle_distances[v] for v in vehicles)
            fitness = fitness_function(total_distance)
            chromosome_fitness.append({
                'chromosome': chrom,
                'assignments': assignments,
                'vehicle_distances': vehicle_distances,
                'total_distance': total_distance,
                'fitness': fitness
            })
        chromosome_fitness.sort(key=lambda x: x['fitness'], reverse=True)
        population = [cd['chromosome'] for cd in chromosome_fitness]

        store_best_chromosomes(generation, chromosome_fitness, best_chromosomes)

        current_best_fitness = chromosome_fitness[0]['fitness']
        if prev_best_fitness is not None and abs(current_best_fitness - prev_best_fitness) < 1e-10:
            unchanged_count += 1
        else:
            unchanged_count = 1
            prev_best_fitness = current_best_fitness

        elitism_count = int(population_size * elitism_rate)
        next_population = population[:elitism_count]
        prev_generation = population[:]

        while len(next_population) < population_size:
            if random.random() < crossover_prob:
                parent1, parent2 = random.sample(population, 2)
                child1, child2 = single_point_crossover(parent1, parent2)
            else:
                child1, child2 = random.sample(population, 2)
            if random.random() < mutation_prob:
                child1 = swap_mutation(child1)
            if random.random() < mutation_prob:
                child2 = swap_mutation(child2)
            child1 = repair_chromosome(child1, customer_parts, vehicles)
            child2 = repair_chromosome(child2, customer_parts, vehicles)

            if is_legitimate_chromosome(child1, customer_parts, vehicles, demands_dict, truck_capacity):
                if child1 not in next_population:
                    next_population.append(child1)
            if len(next_population) < population_size:
                if is_legitimate_chromosome(child2, customer_parts, vehicles, demands_dict, truck_capacity):
                    if child2 not in next_population:
                        next_population.append(child2)

        seen = set(tuple(chrom) for chrom in next_population[:elitism_count])
        unique_population = next_population[:elitism_count]
        i = elitism_count
        while i < len(next_population):
            chrom_tuple = tuple(next_population[i])
            if chrom_tuple not in seen:
                unique_population.append(next_population[i])
                seen.add(chrom_tuple)
            else:
                replaced = False
                for candidate in prev_generation[elitism_count:]:
                    cand_tuple = tuple(candidate)
                    if cand_tuple not in seen:
                        unique_population.append(candidate)
                        seen.add(cand_tuple)
                        replaced = True
                        break
            i += 1
        while len(unique_population) < population_size:
            candidate = random.choice(prev_generation[elitism_count:])
            cand_tuple = tuple(candidate)
            if cand_tuple not in seen:
                unique_population.append(candidate)
                seen.add(cand_tuple)
        population = unique_population[:population_size]

    chromosome_fitness = []
    for chrom in population:
        assignments = parse_chromosome(chrom, vehicles)
        vehicle_distances = calculate_vehicle_distances(
            assignments, customers, demands, vehicles, coordinates
        )
        total_distance = sum(vehicle_distances[v] for v in vehicles)
        fitness = fitness_function(total_distance)
        chromosome_fitness.append({
            'chromosome': chrom,
            'assignments': assignments,
            'vehicle_distances': vehicle_distances,
            'total_distance': total_distance,
            'fitness': fitness
        })

    chromosome_fitness.sort(key=lambda x: x['fitness'], reverse=True)

    print("\nRanked Chromosomes by Fitness (Reciprocal of Total Distance):")
    for rank, chrom_info in enumerate(chromosome_fitness, 1):
        print(f"\nRank {rank}:")
        print(f"  Chromosome: {chrom_info['chromosome']}")
        for v in vehicles:
            print(f"    Vehicle {v} serves: {chrom_info['assignments'][v]}")
        print(f"  Total Distance: {chrom_info['total_distance']:.2f}")
        print(f"  Fitness Value: {chrom_info['fitness']:.6f}")
        for v in vehicles:
            print(f"    Vehicle {v} distance: {chrom_info['vehicle_distances'][v]:.2f}")

    print("\nCustomer Coordinates:")
    for i, coord in enumerate(coordinates, 1):
        print(f"Customer {i}: {coord}")

    print("\nBest Chromosome and Fitness Value of Each Generation:")
    for entry in best_chromosomes:
        print(f"Generation {entry['generation']}: Chromosome = {entry['chromosome']}, Fitness = {entry['fitness']:.6f}")

if __name__ == "__main__":
    genetic_algorithm()


Enter the number of customers:  4
Enter the number of vehicles:  3
Enter demand for customer 1:  16
Enter x-coordinate for customer 1:  23
Enter y-coordinate for customer 1:  -41
Enter demand for customer 2:  19
Enter x-coordinate for customer 2:  -43
Enter y-coordinate for customer 2:  -12
Enter demand for customer 3:  12
Enter x-coordinate for customer 3:  34
Enter y-coordinate for customer 3:  24
Enter demand for customer 4:  14
Enter x-coordinate for customer 4:  30
Enter y-coordinate for customer 4:  9
Enter the load carrying capacity of each truck:  28
Enter the population size:  10
Enter the number of generations with unchanged fitness before stopping:  50
Enter crossover probability (e.g., 0.8):  0.8
Enter elitism rate (e.g., 0.9):  0.1
Enter mutation probability (e.g., 0.2):  0.2



Ranked Chromosomes by Fitness (Reciprocal of Total Distance):

Rank 1:
  Chromosome: ['4', '3', '6', '2', '5', '1', '7']
    Vehicle 5 serves: ['2']
    Vehicle 6 serves: ['4', '3']
    Vehicle 7 serves: ['1']
  Total Distance: 271.77
  Fitness Value: 0.003680
    Vehicle 5 distance: 89.29
    Vehicle 6 distance: 88.46
    Vehicle 7 distance: 94.02

Rank 2:
  Chromosome: ['4', '3', '5', '1', '6', '2', '7']
    Vehicle 5 serves: ['4', '3']
    Vehicle 6 serves: ['1']
    Vehicle 7 serves: ['2']
  Total Distance: 271.77
  Fitness Value: 0.003680
    Vehicle 5 distance: 88.46
    Vehicle 6 distance: 94.02
    Vehicle 7 distance: 89.29

Rank 3:
  Chromosome: ['4', '3', '5', '1', '7', '2', '6']
    Vehicle 5 serves: ['4', '3']
    Vehicle 6 serves: ['2']
    Vehicle 7 serves: ['1']
  Total Distance: 271.77
  Fitness Value: 0.003680
    Vehicle 5 distance: 88.46
    Vehicle 6 distance: 89.29
    Vehicle 7 distance: 94.02

Rank 4:
  Chromosome: ['4', '3', '7', '2', '6', '1', '5']
    Vehicle

In [ ]:
import random
import math
import matplotlib.pyplot as plt

def split_customer_demand(customer_id, demand, capacity):
    splits = []
    while demand > capacity:
        splits.append((f"{customer_id}p{len(splits)+1}", capacity))
        demand -= capacity
    if demand > 0:
        splits.append((f"{customer_id}p{len(splits)+1}", demand))
    return splits

def get_customer_parts(customers, demands, capacity):
    customer_parts = []
    demand_parts = []
    for c, d in zip(customers, demands):
        if d > capacity:
            splits = split_customer_demand(c, d, capacity)
            for label, part_demand in splits:
                customer_parts.append(label)
                demand_parts.append(part_demand)
        else:
            customer_parts.append(str(c))
            demand_parts.append(d)
    return customer_parts, demand_parts

def build_demands_dict(customers, demands, capacity):
    demands_dict = {}
    for c, d in zip(customers, demands):
        if d > capacity:
            splits = split_customer_demand(c, d, capacity)
            for label, part_demand in splits:
                demands_dict[label] = part_demand
        else:
            demands_dict[str(c)] = d
    return demands_dict

def generate_random_chromosome(customer_parts, demand_parts, vehicles, capacity):
    unassigned = list(zip(customer_parts, demand_parts))
    random.shuffle(unassigned)
    vehicles_left = vehicles[:]
    random.shuffle(vehicles_left)
    chromosome = []
    used_vehicles = set()
    i = 0
    n = len(unassigned)
    while i < n and vehicles_left:
        v = vehicles_left.pop()
        used_vehicles.add(v)
        group = []
        load = 0
        while i < n and load + unassigned[i][1] <= capacity:
            group.append(unassigned[i][0])
            load += unassigned[i][1]
            i += 1
        if group:
            chromosome.extend(group)
            chromosome.append(str(v))
    if i < n:
        return None
    for v in vehicles:
        if v not in used_vehicles:
            chromosome.append(str(v))
    return chromosome

def parse_chromosome(chromosome, vehicles):
    vehicle_set = set(str(v) for v in vehicles)
    assignments = {v: [] for v in vehicles}
    i = 0
    n = len(chromosome)
    while i < n:
        group = []
        while i < n and chromosome[i] not in vehicle_set:
            group.append(chromosome[i])
            i += 1
        if i < n and chromosome[i] in vehicle_set:
            v = int(chromosome[i])
            assignments[v].extend(group)
            i += 1
    return assignments

def is_legitimate_chromosome(chrom, customer_parts, vehicles, demands_dict, capacity):
    assigned_customers = set()
    assignments = parse_chromosome(chrom, vehicles)
    for v, cust_list in assignments.items():
        total_load = 0
        for cust in cust_list:
            if cust not in customer_parts or cust in assigned_customers:
                return False
            assigned_customers.add(cust)
            total_load += demands_dict[cust]
        if total_load > capacity:
            return False
    if assigned_customers != set(customer_parts):
        return False
    if len(chrom) != len(set(chrom)):
        return False
    if not all(str(v) in chrom for v in vehicles):
        return False
    return True

def calculate_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def calculate_vehicle_distances(assignments, customers, demands, vehicles, coordinates, depot=(0, 0)):
    cust_part_coords = {}
    for i, c in enumerate(customers):
        cust_part_coords[str(c)] = coordinates[i]
    for i, c in enumerate(customers):
        for part_num in range(2, 20):
            cust_part_coords[f"{c}p{part_num}"] = coordinates[i]
    vehicle_distances = {}
    for v, cust_list in assignments.items():
        if not cust_list:
            vehicle_distances[v] = 0.0
            continue
        total_dist = 0.0
        prev_point = depot
        for cust in cust_list:
            total_dist += calculate_distance(prev_point, cust_part_coords[cust])
            prev_point = cust_part_coords[cust]
        total_dist += calculate_distance(prev_point, depot)
        vehicle_distances[v] = total_dist
    return vehicle_distances

def single_point_crossover(parent1, parent2):
    length = len(parent1)
    if length < 2:
        return parent1[:], parent2[:]
    point = random.randint(1, length - 2)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def swap_mutation(chrom):
    c = chrom[:]
    idx1, idx2 = random.sample(range(len(c)), 2)
    c[idx1], c[idx2] = c[idx2], c[idx1]
    return c

def repair_chromosome(chrom, customer_parts, vehicles):
    seen = set()
    new_chrom = []
    for gene in chrom:
        if gene not in seen:
            new_chrom.append(gene)
            seen.add(gene)
    all_numbers = set(customer_parts + [str(v) for v in vehicles])
    missing = all_numbers - set(new_chrom)
    new_chrom += list(missing)
    return new_chrom[:len(customer_parts) + len(vehicles)]

def fitness_function(total_distance):
    if total_distance == 0:
        return float('inf')  # Best possible
    return 1 / total_distance

def store_best_chromosomes(generation, chromosome_fitness, best_chromosomes):
    best = chromosome_fitness[0]
    best_chromosomes.append({
        'generation': generation,
        'chromosome': best['chromosome'],
        'fitness': best['fitness']
    })

def plot_fitness_vs_generation(best_chromosomes):
    generations = [entry['generation'] for entry in best_chromosomes]
    fitness_values = [entry['fitness'] for entry in best_chromosomes]
    plt.figure(figsize=(10, 6))
    plt.plot(generations, fitness_values, marker='o', linestyle='-', color='b')
    plt.title('Best Fitness Value vs Generation')
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness Value')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def genetic_algorithm():
    num_customers = int(input("Enter the number of customers: "))
    num_vehicles = int(input("Enter the number of vehicles: "))
    customers = []
    demands = []
    coordinates = []
    for i in range(1, num_customers + 1):
        demand = int(input(f"Enter demand for customer {i}: "))
        x = float(input(f"Enter x-coordinate for customer {i}: "))
        y = float(input(f"Enter y-coordinate for customer {i}: "))
        customers.append(i)
        demands.append(demand)
        coordinates.append((x, y))
    vehicles = [i + num_customers for i in range(1, num_vehicles + 1)]
    truck_capacity = int(input("Enter the load carrying capacity of each truck: "))
    population_size = int(input("Enter the population size: "))
    max_unchanged = int(input("Enter the number of generations with unchanged fitness before stopping: "))

    crossover_prob = float(input("Enter crossover probability (e.g., 0.8): "))
    elitism_rate = float(input("Enter elitism rate (e.g., 0.9): "))
    mutation_prob = float(input("Enter mutation probability (e.g., 0.2): "))

    customer_parts, demand_parts = get_customer_parts(customers, demands, truck_capacity)
    demands_dict = build_demands_dict(customers, demands, truck_capacity)

    population = []
    attempts = 0
    while len(population) < population_size and attempts < population_size * 100:
        chrom = generate_random_chromosome(customer_parts, demand_parts, vehicles, truck_capacity)
        if chrom and is_legitimate_chromosome(chrom, customer_parts, vehicles, demands_dict, truck_capacity):
            if chrom not in population:
                population.append(chrom)
        attempts += 1

    if len(population) < population_size:
        print("Could not generate enough legitimate chromosomes for initial population.")
        return

    best_chromosomes = []
    unchanged_count = 0
    prev_best_fitness = None
    generation = 0

    while unchanged_count < max_unchanged:
        generation += 1
        chromosome_fitness = []
        for chrom in population:
            assignments = parse_chromosome(chrom, vehicles)
            vehicle_distances = calculate_vehicle_distances(
                assignments, customers, demands, vehicles, coordinates
            )
            total_distance = sum(vehicle_distances[v] for v in vehicles)
            fitness = fitness_function(total_distance)
            chromosome_fitness.append({
                'chromosome': chrom,
                'assignments': assignments,
                'vehicle_distances': vehicle_distances,
                'total_distance': total_distance,
                'fitness': fitness
            })
        chromosome_fitness.sort(key=lambda x: x['fitness'], reverse=True)
        population = [cd['chromosome'] for cd in chromosome_fitness]

        store_best_chromosomes(generation, chromosome_fitness, best_chromosomes)

        current_best_fitness = chromosome_fitness[0]['fitness']
        if prev_best_fitness is not None and abs(current_best_fitness - prev_best_fitness) < 1e-10:
            unchanged_count += 1
        else:
            unchanged_count = 1
            prev_best_fitness = current_best_fitness

        elitism_count = int(population_size * elitism_rate)
        next_population = population[:elitism_count]
        prev_generation = population[:]

        while len(next_population) < population_size:
            if random.random() < crossover_prob:
                parent1, parent2 = random.sample(population, 2)
                child1, child2 = single_point_crossover(parent1, parent2)
            else:
                child1, child2 = random.sample(population, 2)
            if random.random() < mutation_prob:
                child1 = swap_mutation(child1)
            if random.random() < mutation_prob:
                child2 = swap_mutation(child2)
            child1 = repair_chromosome(child1, customer_parts, vehicles)
            child2 = repair_chromosome(child2, customer_parts, vehicles)

            if is_legitimate_chromosome(child1, customer_parts, vehicles, demands_dict, truck_capacity):
                if child1 not in next_population:
                    next_population.append(child1)
            if len(next_population) < population_size:
                if is_legitimate_chromosome(child2, customer_parts, vehicles, demands_dict, truck_capacity):
                    if child2 not in next_population:
                        next_population.append(child2)

        seen = set(tuple(chrom) for chrom in next_population[:elitism_count])
        unique_population = next_population[:elitism_count]
        i = elitism_count
        while i < len(next_population):
            chrom_tuple = tuple(next_population[i])
            if chrom_tuple not in seen:
                unique_population.append(next_population[i])
                seen.add(chrom_tuple)
            else:
                replaced = False
                for candidate in prev_generation[elitism_count:]:
                    cand_tuple = tuple(candidate)
                    if cand_tuple not in seen:
                        unique_population.append(candidate)
                        seen.add(cand_tuple)
                        replaced = True
                        break
            i += 1
        while len(unique_population) < population_size:
            candidate = random.choice(prev_generation[elitism_count:])
            cand_tuple = tuple(candidate)
            if cand_tuple not in seen:
                unique_population.append(candidate)
                seen.add(cand_tuple)
        population = unique_population[:population_size]

    chromosome_fitness = []
    for chrom in population:
        assignments = parse_chromosome(chrom, vehicles)
        vehicle_distances = calculate_vehicle_distances(
            assignments, customers, demands, vehicles, coordinates
        )
        total_distance = sum(vehicle_distances[v] for v in vehicles)
        fitness = fitness_function(total_distance)
        chromosome_fitness.append({
            'chromosome': chrom,
            'assignments': assignments,
            'vehicle_distances': vehicle_distances,
            'total_distance': total_distance,
            'fitness': fitness
        })

    chromosome_fitness.sort(key=lambda x: x['fitness'], reverse=True)

    print("\nRanked Chromosomes by Fitness (Reciprocal of Total Distance):")
    for rank, chrom_info in enumerate(chromosome_fitness, 1):
        print(f"\nRank {rank}:")
        print(f"  Chromosome: {chrom_info['chromosome']}")
        for v in vehicles:
            print(f"    Vehicle {v} serves: {chrom_info['assignments'][v]}")
        print(f"  Total Distance: {chrom_info['total_distance']:.2f}")
        print(f"  Fitness Value: {chrom_info['fitness']:.6f}")
        for v in vehicles:
            print(f"    Vehicle {v} distance: {chrom_info['vehicle_distances'][v]:.2f}")

    print("\nCustomer Coordinates:")
    for i, coord in enumerate(coordinates, 1):
        print(f"Customer {i}: {coord}")

    print("\nBest Chromosome and Fitness Value of Each Generation:")
    for entry in best_chromosomes:
        print(f"Generation {entry['generation']}: Chromosome = {entry['chromosome']}, Fitness = {entry['fitness']:.6f}")

    # Plot the fitness vs. generation graph
    plot_fitness_vs_generation(best_chromosomes)

if __name__ == "__main__":
    genetic_algorithm()

Enter the number of customers:  3
Enter the number of vehicles:  2
